# Plotting Functions

## Plot horizontal structure

In [3]:
def plot_horizontal_structure(
    frame_to_plot,
    zonal_gridpoints,
    meridional_gridpoints,
    time_points,
    zonal_velocity, 
    meridional_velocity, 
    column_temperature, 
    column_moisture, 
    specified_output_file_directory,
    specified_initial_condition_name,
    physical_parameters,
    simulation_parameters,
    plotting_parameters,
    
):

    import time
    import numpy as np
    import matplotlib.pyplot as plt
    plt.rcParams.update({'font.size':22})
    import matplotlib.ticker as mticker
    import matplotlib.colors as mcolors
    import matplotlib.gridspec as gs

    from tqdm import tqdm
    
    # Auxiliary Functions
    import sys
    sys.path.insert(0, '/home/disk/eos7/sressel/research/thesis-work/python/auxiliary_functions/')
    import ipynb.fs.full.mjo_mean_state_diagnostics as mjo
    from ipynb.fs.full.rounding_functions import round_out, round_to_multiple
    from ipynb.fs.full.modified_colormap import Ahmed21_colormap
    Ahmed_cmap = Ahmed21_colormap()
    from ipynb.fs.full.normalize_data import normalize_data
    
    if frame_to_plot > len(time_points):
        raise ValueError(f"Frame out of range of array size: {frame_to_plot} > {len(time_points)}")

    # Check if there are non-zero temperatures or winds
    temperature_all_zero = not np.any(column_temperature[frame_to_plot])
    wind_all_zero = not (np.any(zonal_velocity[frame_to_plot]) or np.any(meridional_velocity[frame_to_plot]))
    
    [
        SPECIFIC_HEAT, 
        LATENT_HEAT,
        WATER_DENSITY,
        COLUMN_AVERAGE_MASS,
        EARTH_RADIUS,
        METERS_PER_DEGREE,
        SECONDS_PER_DAY,
    ] = physical_parameters

    [
        simulation_moisture,
        fringe_region,
        fringe_region_latitude,
        fringe_region_width
    ] = simulation_parameters
    
    [
        xlims, ylims,
        zonal_quiver_points, meridional_quiver_points,
        save_plot, plotting_units,
        moisture_anomaly_scaling,
        grid_scaling,
        save_timestamp
    ] = plotting_parameters

    # Set plotting options
    plt.style.use('default')
    plt.rcParams.update({'font.size':22})

    # Specify whether to plot the values in simulated units or converted units
    if plotting_units == 'natural':
        plotting_temperature = column_temperature
        temperature_units = r"$\frac{K \, kg}{m^2}$"
        plotting_moisture = column_moisture
        moisture_units = r"$\frac{K \, kg}{m^2}$"

    elif plotting_units == 'converted':
        # Convert the units of temperature and moisture
        average_temperature = np.copy(column_temperature)/COLUMN_AVERAGE_MASS
        equivalent_moisture = np.copy(column_moisture)/LATENT_HEAT*SPECIFIC_HEAT*1000/WATER_DENSITY

        plotting_temperature = average_temperature
        temperature_units = r"K"
        plotting_moisture = equivalent_moisture
        moisture_units = r"mm"

    if (not moisture_anomaly_scaling == 1) and (plotting_units == 'converted'):
        print(f"Plotting note - values are scaled! Maximum moisture anomaly set to {moisture_anomaly_scaling:0.2f} mm \n")
        zonal_velocity *=  (moisture_anomaly_scaling/np.nanmax(np.abs(plotting_moisture[frame_to_plot])))
        meridional_velocity *=  (moisture_anomaly_scaling/np.nanmax(np.abs(plotting_moisture[frame_to_plot])))
        plotting_temperature *=  (moisture_anomaly_scaling/np.nanmax(np.abs(plotting_moisture[frame_to_plot])))
        plotting_moisture *=  (moisture_anomaly_scaling/np.nanmax(np.abs(plotting_moisture[frame_to_plot])))

    ##### Specify arguments for plotting the colors, contours, and quivers
    # This determines the magnitude of the maximum temperature anomaly, and sets the rounding appropriately
    if not temperature_all_zero:
        temperature_order = -int(np.floor(np.log10(np.nanmax(np.abs(plotting_temperature[frame_to_plot]))))-1)
        temperature_levels = np.delete(np.linspace(
                        -round_out(np.nanmax(np.abs(plotting_temperature[frame_to_plot])), temperature_order), 
                         round_out(np.nanmax(np.abs(plotting_temperature[frame_to_plot])), temperature_order), 
                         11
                    ), [5])
        contour_args = {
            'levels' :  temperature_levels,
            'norm'   :  mcolors.CenteredNorm(),
            'colors' :  'black'
        }
    
    if simulation_moisture:
        contourf_args = {
            'levels' :  np.linspace(
                            -round_out(np.nanmax(np.abs(plotting_moisture[frame_to_plot])), 'tenths'),
                             round_out(np.nanmax(np.abs(plotting_moisture[frame_to_plot])), 'tenths'),
                             21
                        ),
            'norm'   :  mcolors.CenteredNorm(),
            'cmap'   : Ahmed_cmap,
            'extend' : 'both'
        }

    if not wind_all_zero:
        quiver_scale = round_out(np.nanmax(np.abs(zonal_velocity[frame_to_plot])), 'tenths')
        # quiver_scale = 1
        # print(f"Quiver scale: {quiver_scale}")
        quiver_args = {
            'color'       : 'k',
            'width'       : 0.0015,
            'width'       : 0.002,
            'angles'      : 'xy',
            'scale_units' : 'xy',
            'scale'       : quiver_scale
        }

    # Find the indices of the left and right bounds of the plot 
    left_bound_index = np.abs(zonal_gridpoints/METERS_PER_DEGREE - xlims[0]).argmin()
    right_bound_index = np.abs(zonal_gridpoints/METERS_PER_DEGREE - xlims[1]).argmin()

    # Calculate the skip number necessary to give 'zonal_quiver_points' number of points
    n_skip_zonal = int(len(zonal_gridpoints[left_bound_index:right_bound_index])/zonal_quiver_points)

    # Find the indices of the bottom and top bounds of the plot 
    bottom_bound_index = np.abs(meridional_gridpoints/METERS_PER_DEGREE - ylims[0]).argmin()
    top_bound_index = np.abs(meridional_gridpoints/METERS_PER_DEGREE - ylims[1]).argmin()

    # Calculate the skip number necessary to give 'meridional_quiver_points' number of points
    n_skip_meridional = int(len(meridional_gridpoints[bottom_bound_index:top_bound_index])/meridional_quiver_points)

    # Create a figure 
    fig = plt.figure(figsize=(16,6),dpi=500)
    
    # If the sim has mositure, add a colorbar
    if simulation_moisture:
        gs_main = gs.GridSpec(1, 2, width_ratios = [100,2], figure=fig)
        qbar_ax = fig.add_subplot(gs_main[-1])
        
    # Without moisture, just one axis is required
    else:
        gs_main = gs.GridSpec(1, 1, figure=fig)
        
    # Update the bounds and spacing of the gs.GridSpec
    gs_main.update(left=0.1, right=0.9, top=0.99, bottom=0.1, wspace=0.05)

    # Add an axis for the initial condition
    ax = fig.add_subplot(gs_main[0])

    # Label the plot
    if frame_to_plot == 0:
        ax.set_title(f"Initial condition", pad=10)
        output_time_string = "initial-condition"
        
    elif time_points[frame_to_plot] <= 120*3600:
        output_time_units = 'hr'
        output_time_string = f"{time_points[frame_to_plot]/(3600):0.1f}-{output_time_units}"
        ax.set_title(f"Time: {time_points[frame_to_plot]/3600:0.1f} {output_time_units}", pad=10)
    
    else:
        output_time_units = 'days'
        output_time_string = f"{time_points[frame_to_plot]/(24*3600):0.1f}-{output_time_units}"
        ax.set_title(f"Time: {time_points[frame_to_plot]/(24*3600):0.1f} {output_time_units}", pad=10)
        
    if not temperature_all_zero:
        # Plot column temperature as a contour
        T_cont = ax.contour(
            zonal_gridpoints*grid_scaling,
            meridional_gridpoints*grid_scaling,
            plotting_temperature[frame_to_plot],
            **contour_args
        )
        ax.clabel(T_cont, T_cont.levels, inline=True, fontsize=10)

    # Plot column moisture as colors
    if simulation_moisture:
        q_cont = plt.contourf(
            zonal_gridpoints*grid_scaling,
            meridional_gridpoints*grid_scaling,
            plotting_moisture[frame_to_plot],
            **contourf_args
        )
        
        cbar = fig.colorbar(
            q_cont, 
            cax=qbar_ax
        )
        cbar.set_ticks(contourf_args['levels'][::4])
        cbar.set_label(moisture_units, rotation=0, labelpad=15)

    if not wind_all_zero:
        # Plot zonal and meridional velocity using vectors
        quiv = ax.quiver(
            zonal_gridpoints[::n_skip_zonal]*grid_scaling,
            meridional_gridpoints[::n_skip_meridional]*grid_scaling,
            -zonal_velocity[frame_to_plot][::n_skip_meridional,::n_skip_zonal],
            -meridional_velocity[frame_to_plot][::n_skip_meridional,::n_skip_zonal],
            **quiver_args
        )
    
        key_label = f"{360/(xlims[1]-xlims[0])*quiver_scale:.2f} m/s"
        key_length = quiver_scale*(xlims[1]-xlims[0])/360

        ax.quiverkey(
            quiv,          
            xlims[1]*METERS_PER_DEGREE*grid_scaling, 1.1*(ylims[1]*METERS_PER_DEGREE*grid_scaling),
            U=key_length,
            label=key_label,
            labelpos='E',          
            color='black', 
            labelcolor='black',
            coordinates='data'
        )

    # Add a dashed line signifying the equator
    ax.axhline(y=0, ls='--', alpha=0.5, color='gray', lw=0.5)

    if fringe_region:
        ax.axhline(y=fringe_region_latitude*METERS_PER_DEGREE*grid_scaling, color='k', ls=':')
        ax.axhline(y=-fringe_region_latitude*METERS_PER_DEGREE*grid_scaling, color='k', ls=':')
        
        ax.axhline(y=(fringe_region_latitude-fringe_region_width)*METERS_PER_DEGREE*grid_scaling, color='k', ls=':')
        ax.axhline(y=-(fringe_region_latitude-fringe_region_width)*METERS_PER_DEGREE*grid_scaling, color='k', ls=':')
    

    ## Set tick labels in longitude/latitude coordinates
    # Longitude
    longitude_ticks = np.arange(xlims[0]+xlims[1]/3, xlims[1]+xlims[1]/3, xlims[1]/3)
    longitude_labels = mjo.tick_labeller(longitude_ticks, direction='lon')
    ax.set_xticks(longitude_ticks*METERS_PER_DEGREE*grid_scaling, labels=longitude_labels)
    
    # Latitude
    latitude_ticks = np.arange(ylims[0], ylims[1]+ylims[1]/3, ylims[1]/3)
    latitude_labels = mjo.tick_labeller(latitude_ticks, direction='lat')
    ax.set_yticks(latitude_ticks*METERS_PER_DEGREE*grid_scaling, labels=latitude_labels)

    # Set the plot's meridional limits and aspect ratio 
    ax.set_xlim(xlims[0]*METERS_PER_DEGREE*grid_scaling,(xlims[1]-1)*METERS_PER_DEGREE*grid_scaling)
    ax.set_ylim(ylims[0]*METERS_PER_DEGREE*grid_scaling,ylims[1]*METERS_PER_DEGREE*grid_scaling)
    ax.set_aspect('auto')
    
    # Save the plot if specified
    if save_plot:
        initial_condition_type = specified_initial_condition_name.split('_')[-1]
        print(f"==============================================")
        print(
            f"Plot saved as:\n" 
            + f"{specified_output_file_directory}/{initial_condition_type}_initial-condition/figures/"
            + f"{specified_initial_condition_name}_{output_time_string}" 
            + f"_horizontal_structure"
            + (f"_{time.strftime('%Y%m%d-%H%M')}" if save_timestamp else '')
            + f".png", 
        )
        plt.savefig(
            f"{specified_output_file_directory}/{initial_condition_type}_initial-condition/figures/"
            + f"{specified_initial_condition_name}_{output_time_string}" 
            + f"_horizontal_structure"
            + (f"_{time.strftime('%Y%m%d-%H%M')}" if save_timestamp else '')
            + f".png", 
            bbox_inches='tight'
        )
    else:
        plt.show()

## Animate solutions

In [5]:
def animate_horizontal_structure(
    zonal_gridpoints,
    meridional_gridpoints,
    time_points,
    zonal_velocity, 
    meridional_velocity, 
    column_temperature, 
    column_moisture, 
    specified_output_file_directory,
    specified_initial_condition_name,
    physical_parameters,
    simulation_parameters,
    plotting_parameters,
    frames,
    normalized_over_time=True
    
):
    #### Imports
    import time
    from tqdm import tqdm
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.animation import FuncAnimation
    import matplotlib.colors as mcolors
    import matplotlib.gridspec as gs

    # Auxiliary Functions
    import sys
    sys.path.insert(0, '/home/disk/eos7/sressel/research/thesis-work/python/auxiliary_functions/')
    import ipynb.fs.full.mjo_mean_state_diagnostics as mjo
    from ipynb.fs.full.rounding_functions import round_out
    from ipynb.fs.full.modified_colormap import Ahmed21_colormap
    Ahmed_cmap = Ahmed21_colormap()

    #### Unpack important parameters
    # Physical parameters
    [
        SPECIFIC_HEAT, 
        LATENT_HEAT,
        WATER_DENSITY,
        COLUMN_AVERAGE_MASS,
        EARTH_RADIUS,
        METERS_PER_DEGREE,
        SECONDS_PER_DAY,
    ] = physical_parameters

    # Experiment parameters
    [
        simulation_moisture,
        fringe_region
    ] = simulation_parameters

    # Plotting parameters
    [
        xlims, ylims,
        zonal_quiver_points, meridional_quiver_points,
        save_plot, plotting_units,
        moisture_anomaly_scaling,
        grid_scaling,
        save_timestamp
    ] = plotting_parameters

    # Specify whether to plot the values in simulated units or converted units
    if plotting_units == 'natural':
        plotting_temperature = column_temperature
        temperature_units = r"$\frac{K \, kg}{m^2}$"
        plotting_moisture = column_moisture
        moisture_units = r"$\frac{K \, kg}{m^2}$"

    elif plotting_units == 'converted':
        # Convert the units of temperature and moisture
        average_temperature = np.copy(column_temperature)/COLUMN_AVERAGE_MASS
        equivalent_moisture = np.copy(column_moisture)/LATENT_HEAT*SPECIFIC_HEAT*1000/WATER_DENSITY

        plotting_temperature = average_temperature
        temperature_units = r"K"
        plotting_moisture = equivalent_moisture
        moisture_units = r"mm"

        # If normalize_over_time is true, rescale the fields at all times to have the same magnitude
        field_scaling_factor = (moisture_anomaly_scaling/np.nanmax(np.abs(plotting_moisture), axis=(1,2)))
        if (normalized_over_time == True):
            print(f"Fields scaled! Maximum moisture anomaly set to {moisture_anomaly_scaling:0.2f} mm")
            zonal_velocity *=  field_scaling_factor[:, None, None]
            meridional_velocity *=  field_scaling_factor[:, None, None]
            plotting_temperature *=  field_scaling_factor[:, None, None]
            plotting_moisture *=  field_scaling_factor[:, None, None]

    # Define the starting frame
    starting_frame = frames[0]
    ending_frame = frames[-1]
    
    ##### Specify arguments for plotting the colors, contours, and quivers
    # Check if there are non-zero temperatures or winds
    temperature_all_zero = not np.any(column_temperature[starting_frame])
    wind_all_zero = not (np.any(zonal_velocity[starting_frame]) or np.any(meridional_velocity[starting_frame]))

    # This determines the magnitude of the maximum temperature anomaly, and sets the rounding appropriately
    if not temperature_all_zero:
        temperature_order = -int(np.floor(np.log10(np.nanmax(np.abs(plotting_temperature[starting_frame]))))-1)
        temperature_levels = np.delete(np.linspace(
                        -round_out(np.nanmax(np.abs(plotting_temperature[starting_frame])), temperature_order), 
                         round_out(np.nanmax(np.abs(plotting_temperature[starting_frame])), temperature_order), 
                         11
                    ), [5])
        contour_args = {
            'levels' :  temperature_levels,
            'norm'   :  mcolors.CenteredNorm(),
            'colors' :  'black'
        }
    
    if simulation_moisture:
        contourf_args = {
            'levels' :  np.linspace(
                            -round_out(np.max(np.abs(plotting_moisture)), 'tenths'),
                             round_out(np.max(np.abs(plotting_moisture)), 'tenths'),
                             21
                        ),
            # 'levels' :  np.linspace(
            #                 -round_out(np.nanmax(np.abs(plotting_moisture[starting_frame])), 'tenths'),
            #                  round_out(np.nanmax(np.abs(plotting_moisture[starting_frame])), 'tenths'),
            #                  21
            #             ),
            'norm'   : mcolors.CenteredNorm(),
            'cmap'   : Ahmed_cmap,
            # 'extend' : 'both'
        }

    if not wind_all_zero:
        quiver_scale = round_out(np.nanmax(np.abs(zonal_velocity[starting_frame])), 'tenths')
        # quiver_scale = round_out(np.nanmax(np.abs(zonal_velocity[starting_frame:ending_frame])), 'tenths')
        quiver_args = {
            'color'       : 'k',
            # 'width'       : 0.0015,
            'width'       : 0.002,
            'angles'      : 'xy',
            'scale_units' : 'xy',
            'scale'       : quiver_scale
        }

    # Find the indices of the left and right bounds of the plot 
    left_bound_index = np.abs(zonal_gridpoints/METERS_PER_DEGREE - xlims[0]).argmin()
    right_bound_index = np.abs(zonal_gridpoints/METERS_PER_DEGREE - xlims[1]).argmin()

    # Calculate the skip number necessary to give 'zonal_quiver_points' number of points
    n_skip_zonal = int(len(zonal_gridpoints[left_bound_index:right_bound_index])/zonal_quiver_points)

    # Find the indices of the bottom and top bounds of the plot 
    bottom_bound_index = np.abs(meridional_gridpoints/METERS_PER_DEGREE - ylims[0]).argmin()
    top_bound_index = np.abs(meridional_gridpoints/METERS_PER_DEGREE - ylims[1]).argmin()

    # Calculate the skip number necessary to give 'meridional_quiver_points' number of points
    n_skip_meridional = int(len(meridional_gridpoints[bottom_bound_index:top_bound_index])/meridional_quiver_points)

    # Set plot style
    plt.style.use('default')
    plt.rcParams.update({'font.size':22})

    # Create a figure 
    fig = plt.figure(figsize=(16,6), dpi=500)

    if simulation_moisture:
        gs_main = gs.GridSpec(1, 2, width_ratios = [100,2], figure=fig)
        gs_main.update(left=0.1, right=0.9, top=0.9, bottom=0.1, wspace=0.05)
    
        gs_maps = gs.GridSpecFromSubplotSpec(1, 1, subplot_spec=gs_main[0])
        gs_cbar = gs.GridSpecFromSubplotSpec(1, 30, subplot_spec=gs_main[1])
        
        ax = fig.add_subplot(gs_maps[0])
        cbar_ax = fig.add_subplot(gs_cbar[:, 1:-1])

    else:
        gs_main = gs.GridSpec(1, 1, figure=fig)
        gs_main.update(left=0.1, right=0.9, top=0.9, bottom=0.1)
        ax = fig.add_subplot(gs_main[:])
        
    def update(frame):
        # Clear the plot each frame
        ax.clear()
        # cbar_ax.clear()
        
        if normalized_over_time:
            normalized_title_string = f", scaled to constant magnitude"
        else:
            normalized_title_string = f""
            
        if time_points[frame] <= 120*3600:
            output_time_units = 'hr'
            output_time_string = f"{time_points[frame]/(3600):0.1f}-{output_time_units}"
            ax.set_title(f"Time: {time_points[frame]/3600:0.1f} {output_time_units}{normalized_title_string}", pad=10)
        
        else:
            output_time_units = 'days'
            output_time_string = f"{time_points[frame]/(24*3600):0.1f}-{output_time_units}"
            ax.set_title(f"Time: {time_points[frame]/(24*3600):0.1f} {output_time_units}{normalized_title_string}", pad=10)

        ##### Specify arguments for plotting the colors, contours, and quivers
        # Check if there are non-zero temperatures or winds
        temperature_all_zero = not np.any(column_temperature[frame])
        wind_all_zero = not (np.any(zonal_velocity[frame]) or np.any(meridional_velocity[frame]))
    
        # This determines the magnitude of the maximum temperature anomaly, and sets the rounding appropriately
        if not temperature_all_zero:
            temperature_order = -int(np.floor(np.log10(np.nanmax(np.abs(plotting_temperature[frame]))))-1)
            temperature_levels = np.delete(
                np.linspace(
                    -round_out(np.nanmax(np.abs(plotting_temperature[frame])), temperature_order), 
                     round_out(np.nanmax(np.abs(plotting_temperature[frame])), temperature_order), 
                     11), [5]
            )
            contour_args = {
                'levels' :  temperature_levels,
                'norm'   :  mcolors.CenteredNorm(),
                'colors' :  'black'
            }

        if not wind_all_zero:
            quiver_scale = round_out(np.nanmax(np.abs(zonal_velocity[frame])), 'tenths')
            quiver_args = {
                'color'       : 'k',
                # 'width'       : 0.0015,
                'width'       : 0.002,
                'angles'      : 'xy',
                'scale_units' : 'xy',
                'scale'       : quiver_scale
            }
        
        # Plot column temperature as a contour
        if not temperature_all_zero:
            T_cont = ax.contour(
                zonal_gridpoints*grid_scaling,
                meridional_gridpoints*grid_scaling,
                plotting_temperature[frame],
                **contour_args
            )
            ax.clabel(T_cont, T_cont.levels, inline=True, fontsize=10)
    
        # Plot column moisture as colors
        if simulation_moisture:
            q_cont = ax.contourf(
                zonal_gridpoints*grid_scaling,
                meridional_gridpoints*grid_scaling,
                plotting_moisture[frame],
                **contourf_args
            )

            cbar = fig.colorbar(
                q_cont, 
                cax=cbar_ax,
                orientation='vertical'
            )
            cbar.set_ticks(contourf_args['levels'][::4])
            cbar.set_label(moisture_units, rotation=0, labelpad=15)

        if not wind_all_zero:
            # Plot zonal and meridional velocity using vectors
            quiv = ax.quiver(
                zonal_gridpoints[::n_skip_zonal]*grid_scaling,
                meridional_gridpoints[::n_skip_meridional]*grid_scaling,
                -zonal_velocity[frame][::n_skip_meridional,::n_skip_zonal],
                -meridional_velocity[frame][::n_skip_meridional,::n_skip_zonal],
                **quiver_args
            )
        
            key_length = quiver_scale*(xlims[1]-xlims[0])/360
            key_label = f"{360/(xlims[1]-xlims[0])*quiver_scale:.2f} m/s"
            
            ax.quiverkey(
                quiv,            
                xlims[1]*METERS_PER_DEGREE*grid_scaling, 1.15*(ylims[1]*METERS_PER_DEGREE*grid_scaling), 
                U=key_length,
                label=key_label,
                labelpos='E',          
                color='black', 
                labelcolor='black',
                coordinates='data'
            )
    
        # Add a dashed line for the equator
        ax.axhline(y=0, ls='--', alpha=0.5, color='gray', lw=0.5)
    
        # Add horizontal lines indicating the fringe region
        if fringe_region:
            ax.axhline(y=fringe_region_latitude*METERS_PER_DEGREE*grid_scaling, color='k', ls=':')
            ax.axhline(y=-fringe_region_latitude*METERS_PER_DEGREE*grid_scaling, color='k', ls=':')
            
            ax.axhline(y=(fringe_region_latitude-fringe_region_width)*METERS_PER_DEGREE*grid_scaling, color='k', ls=':')
            ax.axhline(y=-(fringe_region_latitude-fringe_region_width)*METERS_PER_DEGREE*grid_scaling, color='k', ls=':')
        
        ## Set tick labels in longitude/latitude coordinates
        # Longitude
        longitude_ticks = np.arange(xlims[0]+xlims[1]/3, xlims[1]+xlims[1]/3, xlims[1]/3)
        longitude_labels = mjo.tick_labeller(longitude_ticks, direction='lon')
        ax.set_xticks(longitude_ticks*METERS_PER_DEGREE*grid_scaling, labels=longitude_labels)
        
        # Latitude
        latitude_ticks = np.arange(ylims[0], ylims[1]+ylims[1]/3, ylims[1]/3)
        latitude_labels = mjo.tick_labeller(latitude_ticks, direction='lat')
        ax.set_yticks(latitude_ticks*METERS_PER_DEGREE*grid_scaling, labels=latitude_labels)
    
        # Set the plot's meridional limits and aspect ratio 
        ax.set_xlim(xlims[0]*METERS_PER_DEGREE*grid_scaling,(xlims[1]-1)*METERS_PER_DEGREE*grid_scaling)
        ax.set_ylim(ylims[0]*METERS_PER_DEGREE*grid_scaling,ylims[1]*METERS_PER_DEGREE*grid_scaling)
        # ax.set_aspect('auto')

        # if simulation_moisture:
        #     return T_cont, quiv, q_cont,

        # else:
        #     return T_cont, quiv,

    # Run the animation
    anim = FuncAnimation(fig, update, frames=tqdm(frames, ncols=100, position=0, leave=True), interval=300)

    initial_condition_type = specified_initial_condition_name.split('_')[-1]
    anim.save(
        f"{specified_output_file_directory}/{initial_condition_type}_initial-condition/figures/"
        + f"{specified_initial_condition_name}_animation"
        + (f"_{time.strftime('%Y%m%d-%H%M')}" if save_timestamp else '')
        + f".mp4", 
        dpi=300
    )